In [1]:
import numpy as np
import tensorflow as tf

from sklearn.model_selection import train_test_split

/home/dic/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
def conv_layer(input, channels_in, channels_out,name="conv"):
    with tf.variable_scope(name, reuse=tf.AUTO_REUSE):
        
        #w=tf.Variable(tf.truncated_normal([5,5,channels_in,channels_out], stddev=0.1),name="W")
        w=tf.get_variable('we', shape=(3,3,channels_in,channels_out), initializer=tf.contrib.layers.variance_scaling_initializer()) 

        #initializer=tf.glorot_uniform_initializer()
        #,initializer=tf.truncated_normal_initializer(stddev=0.05)
      #  b=tf.Variable(tf.constant(0.1, shape=[channels_out]), name="B")
        b=tf.get_variable('be', shape=(channels_out), initializer=tf.contrib.layers.variance_scaling_initializer())

        #,initializer=tf.random_uniform_initializer(-0.1,0.1)
        
        conv=tf.nn.conv2d(input,w, strides=[1,1,1,1], padding="SAME")
        act=tf.nn.relu(conv+b)
        tf.summary.histogram("weights", w)
        tf.summary.histogram("biases", b)
        tf.summary.histogram("activations", act)
        return act

def dense_layer(input, channels_in, channels_out,name="dense"):
    with tf.variable_scope(name, reuse=tf.AUTO_REUSE):
        #w=tf.Variable(tf.truncated_normal([channels_in,channels_out], stddev=0.1),name="W")
        w=tf.get_variable('we1', shape=(channels_in,channels_out), initializer=tf.contrib.layers.variance_scaling_initializer())
 
        #b=tf.Variable(tf.constant(0.1, shape=[channels_out]), name="B")
        b=tf.get_variable('be1', shape=(channels_out), initializer=tf.contrib.layers.variance_scaling_initializer())

    
        act=tf.nn.relu(tf.matmul(input, w)+b)
        tf.summary.histogram("weights", w)
        tf.summary.histogram("biases", b)
        tf.summary.histogram("activations", act)
        return act

In [3]:
#setup placeholders
x= tf.placeholder(tf.float32, shape=[None,28,28,1],name="x")#here None will take input at the time of training and it will take the value equal to the batch size 
y= tf.placeholder(tf.float32, shape=[None,7],name="labels")


#Creating the network

#def conv_net(x):
conv1=conv_layer(x, 1, 32,"conv1")
drop1=tf.nn.dropout(conv1,keep_prob=0.1,name="dropout1")
flattened = tf.reshape(drop1, [-1, 28 * 28 * 32])

dense1=dense_layer(flattened,28*28*32,128,"dense1")
drop2=tf.nn.dropout(dense1,keep_prob=0.25,name="dropout2")
dense2=dense_layer(drop2,128,128,"dense2")
logits=dense_layer(dense2,128,7,"dense3")



In [4]:
#pred = conv_net(x)

#cross entropy as loss function
#cost=tf.reduce_mean=(tf.nn.softmax_cross_entropy_with_logits(logits=logits,labels=y))
with tf.name_scope("Loss"):
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y))
    tf.summary.scalar("Loss", cost)
#adamoptimizer
#optimizer=tf.train.AdamOptimizer(0.0002).minimize(cost)
with tf.name_scope("train"):
    optimizer = tf.train.AdamOptimizer(learning_rate=0.0002).minimize(cost)
    
#computing accuracy
with tf.name_scope("accuracy"):
#Here you check whether the index of the maximum value of the predicted image is equal to the actual labelled image. and both will be a column vector.
    correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))

    #calculate accuracy across all the given images and average them out. 
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    tf.summary.scalar("Accuracy", accuracy)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



In [5]:
import numpy as np
#import tensorflow.contrib.eager as tfe
# Load the training data into two NumPy arrays.

classes=7
features = np.load("X_2d.npy")
labels = np.load("y_onehot.npy")
X_train1, X_test1, y_train1, y_test1 = train_test_split(features, labels, test_size=0.2, random_state=1, shuffle=True)
print(y_train1.shape)
X_train = X_train1.reshape(-1, 28, 28, 1)
X_test= X_test1.reshape(-1,28,28,1)
y_train=y_train1.reshape(-1,7)
y_test=y_test1.reshape(-1,7)
print(X_train.shape)
print(y_train)
print(y_train.shape)


epoch=700
batch_size=128
steps=int(len(X_train)/batch_size)

(36504, 7)
(36504, 28, 28, 1)
[[1. 0. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]
 ...
 [0. 0. 1. ... 0. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]]
(36504, 7)


In [6]:

sess=tf.Session()
sess.run(tf.global_variables_initializer())

In [7]:
writer_1 = tf.summary.FileWriter("/home/dic/jupyter/28x28dmdt/log_dir/train_1_700epochs_with_lr_0.0002_variance_scaling_weights_1")
#writer_2 = tf.summary.FileWriter("/home/dic/jupyter/28x28dmdt/log_dir/test_1_200epochs")
writer_1.add_graph(sess.graph)
write_op = tf.summary.merge_all()

In [8]:

train_loss = []
test_loss = []
train_accuracy = []
test_accuracy = []
#writer = tf.summary.FileWriter('/home/dic/jupyter/28x28dmdt/log_dir/1', sess.graph)    
    
for e in range(epoch):
    epoch_loss=0
    
    for batch in range(steps):
        #Calculate our current step
        step = e    *steps + batch
    
        batch_x = X_train[batch*batch_size:min((batch+1)*batch_size,len(X_train))]
        batch_y = y_train[batch*batch_size:min((batch+1)*batch_size,len(y_train))]
        
            #batch,c=sess.run([optimizer,loss],feed_dict={x:batch_x,y=batch_y})
        
        opt = sess.run(optimizer, feed_dict={x: batch_x,y: batch_y})
        loss, acc = sess.run([cost, accuracy], feed_dict={x: batch_x,y: batch_y})
        if step % 100 == 0:
            
            # Get Train Summary for one batch and add summary to TensorBoard                                                      
            summary = sess.run(write_op,feed_dict={x: batch_x,y: batch_y})
            writer_1.add_summary(summary, step)
            writer_1.flush()
    
    print("Iter " + str(e) + ", Loss= ",loss , ", Training Accuracy= ",acc)                                     
    print("Optimization Finished!")
        
       
    test_acc,valid_loss = sess.run([accuracy,cost], feed_dict={x: X_test,y : y_test})
    
    train_loss.append(loss)
    test_loss.append(valid_loss)
    train_accuracy.append(acc)
    test_accuracy.append(test_acc)
    #summary1 = sess.run(write_op,feed_dict={x: X_test,y : y_test})
    #writer_2.add_summary(summary1)
    #writer_2.flush()
    print("Testing Accuracy:",test_acc)
        #print("Testing Accuracy:","{:.5f}".format(test_acc))
        

Iter 0, Loss=  1.4693584 , Training Accuracy=  0.609375
Optimization Finished!
Testing Accuracy: 0.656476
Iter 1, Loss=  1.3190137 , Training Accuracy=  0.640625
Optimization Finished!
Testing Accuracy: 0.6619549
Iter 2, Loss=  1.2728596 , Training Accuracy=  0.65625
Optimization Finished!
Testing Accuracy: 0.6612974
Iter 3, Loss=  1.2929754 , Training Accuracy=  0.6484375
Optimization Finished!
Testing Accuracy: 0.66381764
Iter 4, Loss=  1.2842923 , Training Accuracy=  0.640625
Optimization Finished!
Testing Accuracy: 0.665023
Iter 5, Loss=  1.2941144 , Training Accuracy=  0.6328125
Optimization Finished!
Testing Accuracy: 0.6622836
Iter 6, Loss=  1.2462237 , Training Accuracy=  0.65625
Optimization Finished!
Testing Accuracy: 0.6666667
Iter 7, Loss=  1.227032 , Training Accuracy=  0.6484375
Optimization Finished!
Testing Accuracy: 0.6680912
Iter 8, Loss=  1.1885295 , Training Accuracy=  0.65625
Optimization Finished!
Testing Accuracy: 0.6698444
Iter 9, Loss=  1.1187987 , Training Acc

Testing Accuracy: 0.73679596
Iter 76, Loss=  0.8293079 , Training Accuracy=  0.7109375
Optimization Finished!
Testing Accuracy: 0.73296076
Iter 77, Loss=  0.79590535 , Training Accuracy=  0.7265625
Optimization Finished!
Testing Accuracy: 0.7366864
Iter 78, Loss=  0.8158225 , Training Accuracy=  0.71875
Optimization Finished!
Testing Accuracy: 0.73493314
Iter 79, Loss=  0.7073579 , Training Accuracy=  0.7578125
Optimization Finished!
Testing Accuracy: 0.73109794
Iter 80, Loss=  0.7500839 , Training Accuracy=  0.75
Optimization Finished!
Testing Accuracy: 0.7384396
Iter 81, Loss=  0.86920863 , Training Accuracy=  0.7109375
Optimization Finished!
Testing Accuracy: 0.73942584
Iter 82, Loss=  0.8744024 , Training Accuracy=  0.7265625
Optimization Finished!
Testing Accuracy: 0.7385492
Iter 83, Loss=  0.8262426 , Training Accuracy=  0.7109375
Optimization Finished!
Testing Accuracy: 0.7433706
Iter 84, Loss=  0.87553513 , Training Accuracy=  0.6875
Optimization Finished!
Testing Accuracy: 0.7

Testing Accuracy: 0.7668201
Iter 151, Loss=  0.7352841 , Training Accuracy=  0.765625
Optimization Finished!
Testing Accuracy: 0.7690116
Iter 152, Loss=  0.6907118 , Training Accuracy=  0.75
Optimization Finished!
Testing Accuracy: 0.76638174
Iter 153, Loss=  0.7066176 , Training Accuracy=  0.796875
Optimization Finished!
Testing Accuracy: 0.7690116
Iter 154, Loss=  0.6989225 , Training Accuracy=  0.7578125
Optimization Finished!
Testing Accuracy: 0.76649135
Iter 155, Loss=  0.8726357 , Training Accuracy=  0.7421875
Optimization Finished!
Testing Accuracy: 0.76934034
Iter 156, Loss=  0.71789217 , Training Accuracy=  0.75
Optimization Finished!
Testing Accuracy: 0.76506686
Iter 157, Loss=  0.66282696 , Training Accuracy=  0.765625
Optimization Finished!
Testing Accuracy: 0.77350426
Iter 158, Loss=  0.7269204 , Training Accuracy=  0.7890625
Optimization Finished!
Testing Accuracy: 0.7651764
Iter 159, Loss=  0.75865597 , Training Accuracy=  0.7578125
Optimization Finished!
Testing Accurac

Testing Accuracy: 0.7748192
Iter 226, Loss=  0.61163515 , Training Accuracy=  0.7578125
Optimization Finished!
Testing Accuracy: 0.7783257
Iter 227, Loss=  0.6276053 , Training Accuracy=  0.796875
Optimization Finished!
Testing Accuracy: 0.7819417
Iter 228, Loss=  0.63014376 , Training Accuracy=  0.8046875
Optimization Finished!
Testing Accuracy: 0.78380454
Iter 229, Loss=  0.7371124 , Training Accuracy=  0.7734375
Optimization Finished!
Testing Accuracy: 0.78227043
Iter 230, Loss=  0.6052191 , Training Accuracy=  0.8046875
Optimization Finished!
Testing Accuracy: 0.7874206
Iter 231, Loss=  0.6780014 , Training Accuracy=  0.7734375
Optimization Finished!
Testing Accuracy: 0.78117466
Iter 232, Loss=  0.5765737 , Training Accuracy=  0.8046875
Optimization Finished!
Testing Accuracy: 0.77788734
Iter 233, Loss=  0.6130381 , Training Accuracy=  0.78125
Optimization Finished!
Testing Accuracy: 0.7836949
Iter 234, Loss=  0.6575467 , Training Accuracy=  0.765625
Optimization Finished!
Testing 

Testing Accuracy: 0.7884067
Iter 301, Loss=  0.60931194 , Training Accuracy=  0.8125
Optimization Finished!
Testing Accuracy: 0.78884506
Iter 302, Loss=  0.7096429 , Training Accuracy=  0.765625
Optimization Finished!
Testing Accuracy: 0.7904887
Iter 303, Loss=  0.6490536 , Training Accuracy=  0.78125
Optimization Finished!
Testing Accuracy: 0.7892834
Iter 304, Loss=  0.66916525 , Training Accuracy=  0.7734375
Optimization Finished!
Testing Accuracy: 0.7889546
Iter 305, Loss=  0.615133 , Training Accuracy=  0.8125
Optimization Finished!
Testing Accuracy: 0.78731096
Iter 306, Loss=  0.66767573 , Training Accuracy=  0.8046875
Optimization Finished!
Testing Accuracy: 0.7866535
Iter 307, Loss=  0.55120057 , Training Accuracy=  0.84375
Optimization Finished!
Testing Accuracy: 0.79202276
Iter 308, Loss=  0.60829127 , Training Accuracy=  0.78125
Optimization Finished!
Testing Accuracy: 0.78994083
Iter 309, Loss=  0.6098313 , Training Accuracy=  0.8203125
Optimization Finished!
Testing Accurac

Testing Accuracy: 0.7957484
Iter 376, Loss=  0.6304846 , Training Accuracy=  0.828125
Optimization Finished!
Testing Accuracy: 0.79585797
Iter 377, Loss=  0.5602652 , Training Accuracy=  0.7890625
Optimization Finished!
Testing Accuracy: 0.7954197
Iter 378, Loss=  0.5394293 , Training Accuracy=  0.7890625
Optimization Finished!
Testing Accuracy: 0.79026955
Iter 379, Loss=  0.57868034 , Training Accuracy=  0.8125
Optimization Finished!
Testing Accuracy: 0.7928994
Iter 380, Loss=  0.5811454 , Training Accuracy=  0.78125
Optimization Finished!
Testing Accuracy: 0.79793996
Iter 381, Loss=  0.66812754 , Training Accuracy=  0.78125
Optimization Finished!
Testing Accuracy: 0.79936445
Iter 382, Loss=  0.5732373 , Training Accuracy=  0.8046875
Optimization Finished!
Testing Accuracy: 0.7943239
Iter 383, Loss=  0.6083199 , Training Accuracy=  0.78125
Optimization Finished!
Testing Accuracy: 0.7971729
Iter 384, Loss=  0.50453633 , Training Accuracy=  0.8203125
Optimization Finished!
Testing Accur

Testing Accuracy: 0.799474
Iter 451, Loss=  0.5677173 , Training Accuracy=  0.8203125
Optimization Finished!
Testing Accuracy: 0.7998028
Iter 452, Loss=  0.44274145 , Training Accuracy=  0.8203125
Optimization Finished!
Testing Accuracy: 0.80089855
Iter 453, Loss=  0.5219482 , Training Accuracy=  0.84375
Optimization Finished!
Testing Accuracy: 0.8030901
Iter 454, Loss=  0.482355 , Training Accuracy=  0.8359375
Optimization Finished!
Testing Accuracy: 0.79848784
Iter 455, Loss=  0.5151124 , Training Accuracy=  0.828125
Optimization Finished!
Testing Accuracy: 0.7995836
Iter 456, Loss=  0.561466 , Training Accuracy=  0.78125
Optimization Finished!
Testing Accuracy: 0.8000219
Iter 457, Loss=  0.5370518 , Training Accuracy=  0.84375
Optimization Finished!
Testing Accuracy: 0.8019943
Iter 458, Loss=  0.5112109 , Training Accuracy=  0.828125
Optimization Finished!
Testing Accuracy: 0.8004602
Iter 459, Loss=  0.5695143 , Training Accuracy=  0.796875
Optimization Finished!
Testing Accuracy: 0

Testing Accuracy: 0.80550075
Iter 526, Loss=  0.60356677 , Training Accuracy=  0.8125
Optimization Finished!
Testing Accuracy: 0.80352837
Iter 527, Loss=  0.53843683 , Training Accuracy=  0.8203125
Optimization Finished!
Testing Accuracy: 0.80265176
Iter 528, Loss=  0.65026766 , Training Accuracy=  0.8046875
Optimization Finished!
Testing Accuracy: 0.8034188
Iter 529, Loss=  0.5987023 , Training Accuracy=  0.8046875
Optimization Finished!
Testing Accuracy: 0.8024326
Iter 530, Loss=  0.46540374 , Training Accuracy=  0.8359375
Optimization Finished!
Testing Accuracy: 0.80604863
Iter 531, Loss=  0.5028653 , Training Accuracy=  0.84375
Optimization Finished!
Testing Accuracy: 0.8033092
Iter 532, Loss=  0.52063966 , Training Accuracy=  0.8515625
Optimization Finished!
Testing Accuracy: 0.7990357
Iter 533, Loss=  0.5997066 , Training Accuracy=  0.796875
Optimization Finished!
Testing Accuracy: 0.8053912
Iter 534, Loss=  0.51113456 , Training Accuracy=  0.8515625
Optimization Finished!
Testin

Testing Accuracy: 0.80495286
Iter 601, Loss=  0.5993163 , Training Accuracy=  0.8125
Optimization Finished!
Testing Accuracy: 0.80615824
Iter 602, Loss=  0.5275934 , Training Accuracy=  0.859375
Optimization Finished!
Testing Accuracy: 0.8068157
Iter 603, Loss=  0.5109181 , Training Accuracy=  0.8125
Optimization Finished!
Testing Accuracy: 0.8025422
Iter 604, Loss=  0.579385 , Training Accuracy=  0.8125
Optimization Finished!
Testing Accuracy: 0.8057199
Iter 605, Loss=  0.49973512 , Training Accuracy=  0.828125
Optimization Finished!
Testing Accuracy: 0.8082402
Iter 606, Loss=  0.6045371 , Training Accuracy=  0.8125
Optimization Finished!
Testing Accuracy: 0.8025422
Iter 607, Loss=  0.49706072 , Training Accuracy=  0.8203125
Optimization Finished!
Testing Accuracy: 0.8034188
Iter 608, Loss=  0.57051736 , Training Accuracy=  0.84375
Optimization Finished!
Testing Accuracy: 0.8081306
Iter 609, Loss=  0.6317607 , Training Accuracy=  0.78125
Optimization Finished!
Testing Accuracy: 0.8034

Testing Accuracy: 0.8092264
Iter 676, Loss=  0.53828406 , Training Accuracy=  0.8046875
Optimization Finished!
Testing Accuracy: 0.8104317
Iter 677, Loss=  0.5050231 , Training Accuracy=  0.8359375
Optimization Finished!
Testing Accuracy: 0.80265176
Iter 678, Loss=  0.5368915 , Training Accuracy=  0.84375
Optimization Finished!
Testing Accuracy: 0.8097743
Iter 679, Loss=  0.46155 , Training Accuracy=  0.8359375
Optimization Finished!
Testing Accuracy: 0.8034188
Iter 680, Loss=  0.4887364 , Training Accuracy=  0.8203125
Optimization Finished!
Testing Accuracy: 0.81021255
Iter 681, Loss=  0.5015941 , Training Accuracy=  0.8203125
Optimization Finished!
Testing Accuracy: 0.8053912
Iter 682, Loss=  0.47809157 , Training Accuracy=  0.8359375
Optimization Finished!
Testing Accuracy: 0.8083498
Iter 683, Loss=  0.55948675 , Training Accuracy=  0.8203125
Optimization Finished!
Testing Accuracy: 0.8095551
Iter 684, Loss=  0.54189 , Training Accuracy=  0.796875
Optimization Finished!
Testing Accu

In [10]:
import matplotlib.pyplot as plt
plot_conv_weights(weights=we)

NameError: name 'plot_conv_weights' is not defined